<a href="https://colab.research.google.com/github/athishr88/Autonomous_vehicle/blob/main/LA3/LA3_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install nlp

     |████████████████████████████████| 1.7 MB 7.8 MB/s 
     |████████████████████████████████| 212 kB 70.8 MB/s 


In [5]:
# Imports and dataset loading

import nlp

dataset = nlp.load_dataset('emotion')
train = dataset['train']
val = dataset['validation']
test = dataset['test']

Downloading:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.01k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/204k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/207k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/84e07cd366f4451464584cdbd4958f512bcaddb1e921341e07298ce8a9ce42f4. Subsequent calls will reuse this data.


In [6]:
## Prepare input and output pairs for train dataset
import numpy as np

train_samples = [x['text'] for x in train]
train_labels = [x['label'] for x in train]

## Convert each label in the output to a unique integer
classes = list(set(train_labels))
class_to_index = dict((c,i) for i, c in enumerate(classes))
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])

## Convert the train labels to corresponding int values
train_labels = names_to_ids(train_labels)

In [7]:
## Prep the val dataset
val_samples = [x['text'] for x in val]
val_labels = [x['label'] for x in val]
val_labels = names_to_ids(val_labels)


In [8]:
# Sorting test dataset to short medium and long sentences
import pandas as pd
import math

df = pd.DataFrame(data=test)
s = df.text.str.len().sort_values().index
test_df = df.reindex(s)
test_samples = test_df['text']
test_labels = test_df['label']
test_labels = names_to_ids(test_labels)

test_samples_short = test_samples[0:math.floor(len(test_samples)/3)]
test_labels_short = test_labels[0:math.floor(len(test_samples)/3)]

test_samples_medium = test_samples[math.floor(len(test_samples)/3):math.floor(len(test_samples)*2/3)]
test_labels_medium = test_labels[math.floor(len(test_samples)/3):math.floor(len(test_samples)*2/3)]

test_samples_long = test_samples[math.floor(len(test_samples)*2/3):]
test_labels_long = test_labels[math.floor(len(test_samples)*2/3):]

len(test_labels_long)

667

In [9]:
## Create our Text Vectorizer to index our vocabulary based on the train samples 
from keras.layers import TextVectorization
import tensorflow as tf

vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=100)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128) ## Read batches of 128 samples
vectorizer.adapt(text_ds)

In [10]:
## Print out top five words in the vocab
print(len(vectorizer.get_vocabulary())) ## We set max_tokens=10000
vectorizer.get_vocabulary()[:5]

10000


['', '[UNK]', 'i', 'feel', 'and']

In [11]:
## Create a map to get the unique list of the vocabulary
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))


In [12]:
## Vectorize our data (Convert the string data to integer data)
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()
x_test = vectorizer(np.array([[s] for s in test_samples])).numpy()
x_test_short = vectorizer(np.array([[s] for s in test_samples_short])).numpy()
x_test_medium = vectorizer(np.array([[s] for s in test_samples_medium])).numpy()
x_test_long = vectorizer(np.array([[s] for s in test_samples_long])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)
y_test_short = np.array(test_labels_short)
y_test_medium = np.array(test_labels_medium)
y_test_long = np.array(test_labels_long)

In [ ]:
# # Classifying test set to short, medium and long inputs


# test_samples_sorted = sorted(test_samples, key=len)
# test_samples_short = test_samples_sorted[0:math.floor(len(test_samples_sorted)/3)]
# test_samples_medium = test_samples_sorted[math.floor(len(test_samples_sorted)/3):math.floor(len(test_samples_sorted)*2/3)]
# test_samples_long = test_samples_sorted[math.floor(len(test_samples_sorted)*2/3):]

# # Vectorize


In [14]:
## Download and unzip the Stanford GloVe model (pretrained word embeddings)AA
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-03-07 07:33:31--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-07 07:33:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-07 07:33:31--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [15]:
## Read the embeddings in the pretrained model (we are using the 100D version of GloVe)
import os
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [16]:
## Create "embedding_matrix" to index our vocabulary using the GloVe model 
num_tokens = len(voc) 
embedding_dim = 100 ## 100 dimensions
hits = 0 ## number of words that were found in the pretrained model
misses = 0 ## number of words that were missing in the pretrained model

# Prepare embedding matrix for our word list
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 9627 words (373 misses)


In [17]:
## Define our embedding layer for the training model 
## We load our embedding_matrix as the initializer and set trainable to False to avoid retraining this layer

from keras.layers import Embedding
from keras.initializers import Constant

embedding_layer = Embedding(num_tokens, embedding_dim,
                            embeddings_initializer= Constant(embedding_matrix), 
                            trainable=False,
)

In [19]:
# Training different models
from keras.layers import SimpleRNN, GRU, LSTM
from keras.metrics import Precision, Recall
from keras import layers, Input, Model
from sklearn import metrics

methods_list = ['simple_RNN', 'GRU', 'LSTM']

for method in methods_list:
  int_sequences_input = Input(shape=(None,), dtype="int64")
  embedded_sequences = embedding_layer(int_sequences_input)
  if method == 'simple_RNN':
    x = layers.Bidirectional(layers.SimpleRNN(20, return_sequences=True))(embedded_sequences)
    x = layers.Bidirectional(layers.SimpleRNN(20))(x)
  elif method == 'GRU':
    x = layers.Bidirectional(layers.GRU(20, return_sequences=True))(embedded_sequences)
    x = layers.Bidirectional(layers.GRU(20))(x)
  else:
    x = layers.Bidirectional(layers.LSTM(20, return_sequences=True))(embedded_sequences)
    x = layers.Bidirectional(layers.LSTM(20))(x)
  preds = layers.Dense(len(classes), activation="softmax")(x)
  model = Model(int_sequences_input, preds)
  print(f"RNN method used: {method}")
  model.summary()
  ## Train the model 
  model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])
  model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
  
  # Test
  y_test_pred = model.predict(x_test)
  y_test_pred_labels = np.argmax(y_test_pred, axis=1)

  # Overall Precision
  print(metrics.classification_report(y_test, y_test_pred_labels))

  # Fine grained analysis
  # Short
  print("Fine grained analysis for short tweets")
  y_test_pred = model.predict(x_test_short)
  y_test_pred_labels = np.argmax(y_test_pred, axis=1)
  print(metrics.classification_report(y_test_short, y_test_pred_labels))
  # Medium
  print("Fine grained analysis for Medium length tweets")
  y_test_pred = model.predict(x_test_medium)
  y_test_pred_labels = np.argmax(y_test_pred, axis=1)
  print(metrics.classification_report(y_test_medium, y_test_pred_labels))
  # Long
  print("Fine grained analysis for long tweets")
  y_test_pred = model.predict(x_test_long)
  y_test_pred_labels = np.argmax(y_test_pred, axis=1)
  print(metrics.classification_report(y_test_long, y_test_pred_labels))

RNN method used: simple_RNN
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 40)         4840      
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 40)               2440      
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 6)                 246       
                                                                 
Total params: 1,007,526
Trainab

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       159
           1       0.68      0.32      0.44       275
           2       0.00      0.00      0.00        66
           3       0.50      0.59      0.54       224
           4       0.65      0.83      0.73       581
           5       0.72      0.91      0.80       695

    accuracy                           0.67      2000
   macro avg       0.43      0.44      0.42      2000
weighted avg       0.59      0.67      0.61      2000

Fine grained analysis for short tweets


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.68      0.53      0.59        80
           2       0.00      0.00      0.00        21
           3       0.58      0.66      0.62        85
           4       0.79      0.84      0.82       202
           5       0.74      0.96      0.84       227

    accuracy                           0.73       666
   macro avg       0.47      0.50      0.48       666
weighted avg       0.65      0.73      0.68       666

Fine grained analysis for Medium length tweets


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        46
           1       0.69      0.31      0.43        99
           2       0.00      0.00      0.00        23
           3       0.51      0.67      0.58        67
           4       0.67      0.82      0.74       192
           5       0.74      0.93      0.82       240

    accuracy                           0.68       667
   macro avg       0.43      0.45      0.43       667
weighted avg       0.61      0.68      0.63       667

Fine grained analysis for long tweets


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        62
           1       0.70      0.17      0.27        96
           2       0.00      0.00      0.00        22
           3       0.40      0.43      0.41        72
           4       0.53      0.82      0.65       187
           5       0.68      0.83      0.75       228

    accuracy                           0.59       667
   macro avg       0.39      0.38      0.35       667
weighted avg       0.53      0.59      0.52       667

RNN method used: GRU
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 bidirectional_4 (Bidir